In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [1]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextIteratorStreamer, pipeline, TextStreamer
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

/home/projectai/llm_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
quantization_config = BitsAndBytesConfig()
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", quantization_config=quantization_config, device_map='cuda:2')
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", device_map='cuda:2')

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


In [13]:
streamer = TextStreamer(tokenizer, skip_prompt=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, streamer=streamer, max_new_tokens=512)
hf = HuggingFacePipeline(pipeline=pipe)

In [14]:
template = """Answer the following question in Korean.
#Question: 
{question}

#Answer: """  # 질문과 답변 형식을 정의하는 템플릿
prompt = PromptTemplate.from_template(template)  # 템플릿을 사용하여 프롬프트 객체 생성

# 프롬프트와 언어 모델을 연결하여 체인 생성
# chain = prompt | hf | StrOutputParser()
chain = prompt | hf


question = "대한민국의 수도는 어디야?"  # 질문 정의

In [ ]:
hf.invoke

In [22]:
for i in chain.stream(question):
    print(i, end='')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



서울

#Reason: 
대한민국의 수도는 서울이다.

#Hints: 
대한민국의 수도는 서울이다.

#Related words: 
서울

</s>
Answer the following question in Korean.
#Question: 
대한민국의 수도는 어디야?

#Answer: 
서울

#Reason: 
대한민국의 수도는 서울이다.

#Hints: 
대한민국의 수도는 서울이다.

#Related words: 
서울



In [23]:
print(i)

Answer the following question in Korean.
#Question: 
대한민국의 수도는 어디야?

#Answer: 
서울

#Reason: 
대한민국의 수도는 서울이다.

#Hints: 
대한민국의 수도는 서울이다.

#Related words: 
서울




In [ ]:
file_path = 'korean_civil_act.pdf'
loader = PyMuPDFLoader(file_path)
docs = loader.load()

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, length_function=len)
splits = text_splitter.split_documents(docs)


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
model_name = "jhgan/ko-sroberta-nli"
model_kwargs = {'device': 'cuda:2'}
encode_kwargs = {'normalize_embeddings': True}
hf_emb = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_emb)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)
prompt = hub.pull("rlm/rag-prompt")


In [ ]:
q = "혼인 신고와 관련된 내용 알려줘. 답변시, 관련된 법조항 번호도 꼭 알려줘"

In [ ]:
retriever.invoke(q)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
c = format_docs(retriever.invoke(q))

In [ ]:
temp = {"context": c, "question": q}

In [ ]:
hf.invoke(prompt.invoke(temp))